### Step 7: Frequency Model Fit – GLM Models

We now begin the modeling phase for **claim frequency**, using the processed and feature-engineered dataset (*df_freq_sev_v2*). Our objective is to understand the key drivers of claim counts and to construct the first component of the **pure premium** calculation.

Given the **count-based nature** of our target variable (*ClaimNb_cap*), we adopt a **Generalized Linear Model (GLM)** framework suitable for modeling discrete events.

#### Modeling Roadmap:
- **GLM Poisson**: as the simplest model assuming equal mean and variance.
- **GLM Negative Binomial (NB)**: to allow overdispersion (variance > mean), a common feature in insurance data.

#### Key Technical Notes:
- An **offset term** *log(Exposure_cap)* is included in both models to normalize claim counts over differing policy durations.
- Predictors used are those refined during preprocessing: transformed continuous variables, binned categorical features, and selected interaction terms.
- Model comparison relied on AIC, BIC, deviance, and predictive performance metrics such as RMSE and MAE.

This modeling step provides a foundational understanding of claim frequency, with a strong focus on balancing predictive power, interpretability, and computational feasibility.

In [1]:
import pandas as pd

df_freq_sev_v2 = pd.read_csv('../data/df_freq_sev_v2.csv')

# Data conversions
for col in df_freq_sev_v2.select_dtypes(include='object').columns:
    df_freq_sev_v2[col] = df_freq_sev_v2[col].astype('category')

#### Step 7.A: Initial Modelling approach

In [2]:
import numpy as np

# Main Formula
formula = (
    'ClaimNb_cap ~ VehAge_log + I(VehAge_log**2) + DrivAge_cap + I(DrivAge_cap**2) + '
    'C(VehPower_bin) + Density_log + C(BonusMalus_bin) + C(Area_bin) +'
    'C(VehBrand_bin) + C(VehGas) + C(BonusMalus_bin):C(Region_bin)'
)

In [3]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm

# GLM formula (Poisson and Negative Binomial)
formula_glm = formula

# Matrix
y_glm, X_glm = patsy.dmatrices(formula_glm, data=df_freq_sev_v2, return_type='dataframe')

# Storage of results
results_summary = {}

# 1. GLM Poisson 
glm_poisson = sm.GLM(
    y_glm,
    X_glm,
    family=sm.families.Poisson(),
    offset=df_freq_sev_v2['exposure_log']
)
res_poisson = glm_poisson.fit()

y_pred_poisson = res_poisson.predict()

# 2. GLM Negative Binomial
glm_nb = sm.GLM(
    y_glm,
    X_glm,
    family=sm.families.NegativeBinomial(),
    offset=df_freq_sev_v2['exposure_log']
)
res_nb = glm_nb.fit()

y_pred_nb = res_nb.predict()

# Metrics function
def compute_metrics(model_result, y_true, y_pred, model_name):
    pearson_chi2 = np.sum((y_true - y_pred) ** 2 / (y_pred + 1e-10))

    with np.errstate(divide='ignore', invalid='ignore'):
        deviance = 2 * np.nansum(y_true * np.log((y_true + 1e-10) / (y_pred + 1e-10)) - (y_true - y_pred))

    return {
        'No. Observations': len(y_true),
        'Df Residuals': model_result.df_resid,
        'Df Model': model_result.df_model,
        'Log-Likelihood': round(model_result.llf, 2),
        'AIC': round(model_result.aic, 2),
        'BIC': round(model_result.bic_llf, 2),
        'Deviance': round(deviance, 2),
        'Pearson chi-squared': round(pearson_chi2, 2),
        'Predicted Zero Claims': int(np.sum(y_pred < 0.5)),
        'Actual Zero Claims': int(np.sum(y_true == 0)),
    }

# Metrics
y_true = y_glm['ClaimNb_cap']

results_summary['GLM Poisson'] = compute_metrics(res_poisson, y_true, y_pred_poisson, 'Poisson')
results_summary['GLM Negative Binomial'] = compute_metrics(res_nb, y_true, y_pred_nb, 'Negative Binomial')

# Results
summary_df = pd.DataFrame(results_summary).T.reset_index().rename(columns={'index': 'Model'})
print(summary_df.to_string(index=False))


c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                Model  No. Observations  Df Residuals  Df Model  Log-Likelihood       AIC       BIC  Deviance  Pearson chi-squared  Predicted Zero Claims  Actual Zero Claims
          GLM Poisson          678013.0      677973.0      39.0      -142133.28 284346.55 284803.63 214950.12           1577364.00               677813.0            643953.0
GLM Negative Binomial          678013.0      677973.0      39.0      -141815.26 283710.53 284167.61 214965.07           1554285.96               677780.0            643953.0


#### Step 7.A Summary: Initial Frequency Modeling (*ClaimNb_cap*)

We modeled motor claim frequency using two distinct statistical approaches, focusing on the number of claims per policy (capped at 4) and incorporating exposure as an offset in both models to properly account for differing policy durations.

The models evaluated were:

- **GLM Poisson**  
- **GLM Negative Binomial (NB)**  

The modeling formula included:

- **Continuous predictors**: *VehAge_log*, *I(VehAge_log²)*, *DrivAge_cap*, *I(DrivAge_cap²)*, *Density_log*  
- **Categorical predictors**: *VehPower_bin*, *VehGas*, *VehBrand_bin*, *BonusMalus_bin*, *Area_bin*, and the interaction term *BonusMalus_bin : Region_bin*

##### Modeling Objectives and Rationale

- The **Poisson model** serves as a standard benchmark but assumes equal mean and variance, which rarely holds in insurance data.
- The **GLM Negative Binomial** model allows for overdispersion by assuming a variance function greater than the mean. While the dispersion parameter is not explicitly estimated here, this formulation captures key variability in claim count data.

##### Key Insights

- All models were fit on the full dataset (678,013 records).
- The **GLM Negative Binomial model** demonstrated superior performance (lowest AIC and BIC), while being computationally efficient and easy to deploy.
- NB-based models significantly outperformed Poisson in terms of deviance and Pearson χ², confirming overdispersion in the data.

--

##### Recommendation

The **GLM Negative Binomial model** is selected as the production-ready choice due to:

- Strong portfolio-level fit
- Simplicity and interpretability
- Compatibility with severity modeling (also GLM-based)

This model strikes an excellent balance between accuracy, robustness, and feasibility—making it ideal for integration into the pure premium calculation.

--

##### Additional Notes for Future Development

While not retained in the final pipeline, other model families were explored during development:

- **Discrete Negative Binomial (CountModel NB2)**: Initially tested for more flexible dispersion estimation but removed due to poor compatibility with the exposure offset and less stable convergence behavior.
- **Zero-Inflated Poisson (ZIP)** and **Zero-Inflated Negative Binomial (ZINB)**: Explored to address the high proportion of zero claims (~95%). However, despite careful tuning, ZINB failed to consistently outperform GLM NB in terms of AIC and stability, and was ultimately excluded to prioritize interpretability and reproducibility.

These more advanced models remain valuable for future experiments, particularly in portfolios with more extreme zero inflation or structural zeros (e.g., young policies or niche vehicle types).

---

#### Step 7.B: Train-Test Validation and Model Performance

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.formula.api as smf
import numpy as np

# Split data
df_train, df_test = train_test_split(df_freq_sev_v2, test_size=0.2, random_state=42)

# Formula
formula = (
    'ClaimNb_cap ~ VehAge_log + I(VehAge_log**2) + DrivAge_cap + I(DrivAge_cap**2) + '
    'C(VehPower_bin) + Density_log + C(BonusMalus_bin) + C(Area_bin) + '
    'C(VehBrand_bin) + C(VehGas) + C(BonusMalus_bin):C(Region_bin)'
)

# Fit model on training data
model_nb = smf.glm(formula=formula,
                   data=df_train,
                   family=sm.families.NegativeBinomial(),
                   offset=df_train['exposure_log']).fit()

# Predict on test
df_test['freq_pred'] = model_nb.predict(df_test, offset=df_test['exposure_log'])

# Evaluate performance
mse = mean_squared_error(df_test['ClaimNb_cap'], df_test['freq_pred'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_test['ClaimNb_cap'], df_test['freq_pred'])

print("*** Frequency Model Validation ***")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Mean actual: {df_test['ClaimNb_cap'].mean():.4f}")
print(f"Mean predicted: {df_test['freq_pred'].mean():.4f}")
print(f"Zero claims - actual: {(df_test['ClaimNb_cap'] == 0).mean()*100:.2f}%")
print(f"Zero claims - predicted: {(df_test['freq_pred'] < 0.5).mean()*100:.2f}%")

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


*** Frequency Model Validation ***
RMSE: 0.2371
MAE: 0.0992
Mean actual: 0.0534
Mean predicted: 0.0537
Zero claims - actual: 94.95%
Zero claims - predicted: 99.97%


#### Step 7.B Summary: Frequency Model Validation – Train/Test Split (GLM NB)

We validated the selected **GLM Negative Binomial** model on a hold-out test set (20% of the dataset, 135,603 policies) to assess its generalization performance.

- The model **predicts the overall average claim frequency very accurately**, indicating strong alignment with portfolio-level risk.
- The **MAE and RMSE are within expected ranges** for motor insurance frequency data, which is typically sparse and overdispersed.
- As expected, the model does not capture the individual-level 0-claim behavior well; it predicts nearly all policies as having negligible claim frequency, causing **over-smoothing near zero**.

##### Conclusion:
Despite its limitations in predicting individual zeros, the GLM NB model is retained for production due to its:
- Strong performance at portfolio level
- Simplicity, interpretability, and speed
- Compatibility with the chosen GLM framework used in severity modeling

This validation step confirms the model is reliable for pricing purposes, particularly when combined with claim severity to estimate the **pure premium**.

---

#### Step 7.C: Refit Negative Binomial GLM on full dataset

In [5]:
import statsmodels.formula.api as smf

# Final Refit
model_nb_full = smf.glm(
    formula=formula,
    data=df_freq_sev_v2,
    family=sm.families.NegativeBinomial(),
    offset=df_freq_sev_v2['exposure_log']
).fit()

# Generate final predictions for all rows
df_freq_sev_v2['freq_pred'] = model_nb_full.predict(df_freq_sev_v2, offset=df_freq_sev_v2['exposure_log'])

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


#### Step 7.D: Final Model Coefficients – GLM Negative Binomial

The following table shows the estimated coefficients from the final frequency model (GLM NB). These represent the *log-effect* of each variable on the expected claim count (ClaimNb), compared to a reference level (usually the first category).

In [6]:
import pandas as pd

# Coefficients
coeffs_nb = model_nb_full.params.sort_values()
coeffs_nb = coeffs_nb.reset_index()
coeffs_nb.columns = ['Variable', 'Coefficient']

# Display
pd.set_option('display.max_rows', 100)
coeffs_nb

,Variable,Coefficient
0,C(BonusMalus_bin)[Extreme_150plus]:C(Region_bi...,-20.710103
1,C(BonusMalus_bin)[Extreme_150plus]:C(Region_bi...,-20.449762
2,Intercept,-1.936463
3,VehAge_log,-1.020591
4,C(BonusMalus_bin)[T.Safe_50],-0.785996
5,C(BonusMalus_bin)[Safe_50]:C(Region_bin)[T.Low...,-0.402576
6,C(BonusMalus_bin)[T.Standard_55_75],-0.344676
7,C(BonusMalus_bin)[Extreme_150plus]:C(Region_bi...,-0.325365
8,C(BonusMalus_bin)[High_100_150]:C(Region_bin)[...,-0.217292
9,C(VehBrand_bin)[T.Group_Other],-0.216891


#### Step 7.D Summary: Interpretation of Frequency Model Coefficients (GLM Negative Binomial)

To enhance interpretability and decision-making, we now analyze the coefficients obtained from the final GLM Negative Binomial model fitted on the full dataset. These coefficients express the **log-relative change in expected claim frequency** compared to a reference category (typically the most frequent or standard level for each variable).

--

##### Key Insights

- **Intercept**: Represents the baseline log-claim frequency for a policyholder in all reference categories.
- **VehAge_log**: Has a **strong negative effect** (−1.02), indicating that **older vehicles are less likely to generate claims**.
- **DrivAge_cap**: Shows a small but **positive effect**, meaning **older drivers** are associated with a slightly **higher frequency**.
- **Density_log**: Positively associated with claim frequency, suggesting that **urban areas (higher density)** carry **greater claim risk**.

--

##### Notable Categorical Effects

- `BonusMalus_bin`:
  - **Extreme_150plus**: Very strong **positive impact** (**+1.29**), confirming that drivers with the worst Bonus-Malus ratings are the **most likely to file a claim**.
  - **T.Safe_50**: Strong **negative effect** (**−0.78**), reflecting **safer driving behavior**.
  
- `VehPower_bin`:
  - **Mid_Volume_High_Risk**: Moderate **positive effect**, indicating that vehicles in this category tend to be **riskier**.
  - **Very_High_Volume**: Also contributes positively to claim frequency.

- `Area_bin`:
  - As expected, riskier areas (e.g., **Medium Risk**, **Med_High_Risk**) are associated with **higher predicted frequency**.

--

##### Interaction Effects

The model includes interactions between *BonusMalus_bin* and *Region_bin*, capturing regional variation in driving behavior:

- Some combinations, such as **Extreme BonusMalus in Low Risk regions**, yield **very large negative coefficients** (e.g., −20), likely due to **rare data combinations**. These may be unstable and should be reviewed for volume before interpretation.
- Other interactions refine the impact of *BonusMalus_bin* by regional context, showing more granular segmentation opportunities.

--

##### Practical Interpretation Example

Consider a policyholder with the following characteristics:

- **BonusMalus = 160** → Categorized as *Extreme_150plus*
- **Vehicle power = 10** → Falls under *Mid_Volume_High_Risk*
- **Region = R11** → Part of the *High_Risk* region group
- **VehBrand = B11** → Belongs to the *Group_Risky* brand segment

This combination of **high risk driving history**, **moderate-to-high vehicle power**, and **location/vehicle type associated with above-average claims** is associated with a **substantially higher predicted claim frequency** compared to a baseline profile (e.g., a driver with BonusMalus = 50, low vehicle power, living in R83 with a car brand like B2).

--

These findings will guide the construction of risk-based tariffs and refinement of pricing strategies.

---

In [7]:
df_freq_sev_v2.to_csv('../data/df_freq_sev_v2.csv', index=False)